In [3]:
# import libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

import time

import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [4]:
# import dataset (comes with colab!)
data = np.loadtxt(open('../FFNs/mnist_train_small.csv','rb'),delimiter=',')

# extract labels (number IDs) and remove from data
labels = data[:,0]
data   = data[:,1:]

# normalize the data to a range of [0 1]
dataNorm = data / np.max(data)

In [5]:
# Step 1: convert to tensor
dataT   = torch.tensor( dataNorm ).float()
labelsT = torch.tensor( labels ).long()

# Step 2: use scikitlearn to split the data
train_data,test_data, train_labels,test_labels = train_test_split(dataT, labelsT, test_size=.1)


# Step 3: convert into PyTorch Datasets
train_data = torch.utils.data.TensorDataset(train_data,train_labels)
test_data  = torch.utils.data.TensorDataset(test_data,test_labels)

# Step 4: translate into dataloader objects
batchsize  = 32
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

In [6]:
# create a class for the model
def createTheMNISTNet():

  class mnistNet(nn.Module):
    def __init__(self):
      super().__init__()

      ### input layer
      self.input = nn.Linear(784,64)
      
      ### hidden layer
      self.fc1 = nn.Linear(64,32)
      self.fc2 = nn.Linear(32,32)

      ### output layer
      self.output = nn.Linear(32,10)

    # forward pass
    def forward(self,x):
      x = F.relu( self.input(x) )
      x = F.relu( self.fc1(x) )
      x = F.relu( self.fc2(x) )
      return self.output(x)
  
  # create the model instance
  net = mnistNet()
  
  # loss function
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.Adam(net.parameters(),lr=.01)

  return net,lossfun,optimizer

In [13]:
def function2trainTheModel():

  # Start the timer
  timerInFunction = time.process_time()

  # number of epochs
  numepochs = 10
  
  # create a new model
  net,lossfun,optimizer = createTheMNISTNet()

  # initialize losses
  losses    = torch.zeros(numepochs)
  trainAcc  = []
  testAcc   = []


  # loop over epochs
  for epochi in range(numepochs):

    # loop over training data batches
    batchAcc  = []
    batchLoss = []
    for X,y in train_loader:

      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())

      # compute accuracy
      matches = torch.argmax(yHat,axis=1) == y     # booleans (false/true)
      matchesNumeric = matches.float()             # convert to numbers (0/1)
      accuracyPct = 100*torch.mean(matchesNumeric) # average and x100
      batchAcc.append( accuracyPct )               # add to list of accuracies
    # end of batch loop...

    # now that we've trained through the batches, get their average training accuracy
    trainAcc.append( np.mean(batchAcc) )

    # and get average losses across the batches
    losses[epochi] = np.mean(batchLoss)

    # test accuracy
    X,y = next(iter(test_loader)) # extract X,y from test dataloader
    with torch.no_grad(): # deactivates autograd
      yHat = net(X)
      
    # compare the following really long line of code to the training accuracy lines
    testAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

    # Finally, report the epoch number, computation time, and accuracy
    comptime = time.process_time() - timerInFunction
    print(f'Epoch {epochi+1}/{numepochs}, elapsed time: {comptime:.2f} milli second, test accuracy {testAcc[-1]:.0f}%')

  # end epochs

  # function output
  return trainAcc,testAcc,losses,net

In [14]:
trainAcc,testAcc,losses,net = function2trainTheModel()

Epoch 1/10, elapsed time: 17.97 milli second, test accuracy 91%
Epoch 2/10, elapsed time: 35.12 milli second, test accuracy 93%
Epoch 3/10, elapsed time: 52.02 milli second, test accuracy 94%
Epoch 4/10, elapsed time: 69.11 milli second, test accuracy 94%
Epoch 5/10, elapsed time: 86.73 milli second, test accuracy 94%
Epoch 6/10, elapsed time: 103.11 milli second, test accuracy 94%
Epoch 7/10, elapsed time: 122.97 milli second, test accuracy 94%
Epoch 8/10, elapsed time: 139.38 milli second, test accuracy 94%
Epoch 9/10, elapsed time: 156.22 milli second, test accuracy 93%
Epoch 10/10, elapsed time: 173.44 milli second, test accuracy 94%


In [16]:
# now run a second timer over repeated iterations

# Start the timer (note the different variable name)
timerOutsideFunction = time.process_time()

for i in range(10):
  function2trainTheModel()

TotalExperimentTime = time.process_time() - timerOutsideFunction
print(f'\n\n\nTotal elapsed experiment time: {TotalExperimentTime/60:.3f} seconds')

Epoch 1/10, elapsed time: 17.97 milli second, test accuracy 92%
Epoch 2/10, elapsed time: 36.67 milli second, test accuracy 93%
Epoch 3/10, elapsed time: 53.70 milli second, test accuracy 94%
Epoch 4/10, elapsed time: 71.08 milli second, test accuracy 94%
Epoch 5/10, elapsed time: 87.70 milli second, test accuracy 95%
Epoch 6/10, elapsed time: 105.16 milli second, test accuracy 94%
Epoch 7/10, elapsed time: 121.83 milli second, test accuracy 93%
Epoch 8/10, elapsed time: 139.19 milli second, test accuracy 93%
Epoch 9/10, elapsed time: 156.42 milli second, test accuracy 94%
Epoch 10/10, elapsed time: 173.52 milli second, test accuracy 95%
Epoch 1/10, elapsed time: 16.75 milli second, test accuracy 91%
Epoch 2/10, elapsed time: 33.52 milli second, test accuracy 94%
Epoch 3/10, elapsed time: 51.08 milli second, test accuracy 93%
Epoch 4/10, elapsed time: 68.39 milli second, test accuracy 95%
Epoch 5/10, elapsed time: 84.42 milli second, test accuracy 94%
Epoch 6/10, elapsed time: 101.75 m